In [2]:
import os, shutil, gdal
import numpy as np
import rasterio as rio
import geopandas as gpd
from shapely.geometry import box
from rasterio.features import rasterize

from math import pi

from skimage.morphology import square,diamond,ball, disk
from skimage.measure import regionprops, label

from scipy.ndimage.filters import generic_filter

from numba import njit

from datetime import datetime

from joblib import Parallel, delayed

from scipy import ndimage as ndi
from skimage.morphology import erosion, dilation, opening, closing, white_tophat, binary_erosion, remove_small_objects

# Get list of DSMS

In [ ]:
loc_dsm = '../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/'
dsms = [os.path.join(loc_dsm,f) for f in os.listdir(loc_dsm) if f.endswith(".tif")]

#overviews = Parallel(n_jobs=10, verbose=30)(delayed(developOverviews)(dsm) for dsm in dsms)

# Get Roads in EPC Area

In [ ]:
with rio.open("../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/EPC_DSM80cm_2019.vrt") as src:
    full_bounds = src.bounds
    full_bb = box(full_bounds.left,full_bounds.bottom, full_bounds.right, full_bounds.top)
    

if "routes_pc" not in globals():
    osm_lines_loc = "../OtherData/arizona-latest-20200507.shp/gis_osm_roads_free_1.shp"
    routes = gpd.read_file(osm_lines_loc).to_crs('epsg:2868')
    routes_pc = gpd.clip(routes, full_bb)

# Calculate Slope

Don't need this?

# Calculate TRI

In [ ]:
@njit(fastmath=True)
def windowTRI(a):
    shp = a.shape
    mm = round(a.shape[0]/2)
    centralVal = a[mm]
    diff = np.absolute(a-centralVal)
    average = np.mean(diff)
    total_size = shp[0]
    tri = average*(total_size)/(total_size-1)
    return tri

def computeTRI(arr, winsize):
    if winsize % 2 == 0:
        raise ValueError("Bad window size. Must be odd number")
    out = generic_filter(arr, windowTRI, footprint=square(winsize), mode='mirror', cval=0)
    return out

#tri_test = computeTRI(test_array, 5)
#tri_test[2,2]
def createTRIRasters(infile, out_loc, window_size=5, overwrite=True):
    ofile = os.path.join(out_loc, os.path.basename(infile).replace(".tif","_TRI.tif"))
    if os.path.exists(ofile) and overwrite == False:
        return ofile
    
    with rio.open(infile) as ras:
        msk = ras.read_masks(1)
        dsm_array = ras.read(1)
        dsm_array[msk==0] = np.NaN
        kwargs = ras.profile

    tri = computeTRI(dsm_array, window_size)
    tri[msk==0] = np.NaN
    kwargs.update(tiled=True, compress='lzw')
    with rio.open(ofile, 'w', **kwargs) as oras:
        oras.write(tri, 1)                 
    
    return ofile

In [ ]:
tri_loc = "../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm_TRI/"
os.makedirs(tri_loc, exist_ok=True)
tri_files = Parallel(n_jobs=10, verbose=5)(delayed(createTRIRasters)(dsm, tri_loc, window_size=5, overwrite=False) for dsm in dsms)
gdalbuildvrt_cmd = f"""gdalbuildvrt {os.path.join(tri_loc,'EPC_DSM80cmTRI_2019.vrt')} {tri_loc}/*.tif"""
os.system(gdalbuildvrt_cmd)

_____________________


# Calculate Slope for 30m DEMs from NED

In [ ]:
ned_tifs = [os.path.join("../OtherData/elevation_NED30M/", os.path.basename(f)) for f in os.listdir("../OtherData/elevation_NED30M/") if f.endswith(".tif")]
ned_slope_dir = "../OtherData/elevation_NED30M_Slope"
os.makedirs(ned_slope_dir, exist_ok=True)
slopes = Parallel(n_jobs=4, verbose=5)(delayed(calcSlopes)(ned_tif, ned_slope_dir, overwrite=False) for ned_tif in ned_tifs)
gdalbuildvrt_cmd = f"""gdalbuildvrt {os.path.join(ned_slope_dir,'NED30m_Slope.vrt')} {' '.join(slopes)}"""
os.system(gdalbuildvrt_cmd)

1. Low TRI mean flatter, but could be a building rooftop or ground
2. For building rooftops, they are characterized by high variance of TRI across the larger radius radius. 300ft would be maxium radial size of building (600 ft span)

In [ ]:
@njit(fastmath=True)
def windowStd(a):
    return np.std(a)

def getStdDevWindow(arr, winsize):
    out = generic_filter(arr, windowStd, footprint=diamond(winsize), mode='mirror')
    return out

loc_tri = '../EPCExtent_30cm/Elevation_80cmNPS/DSM_TRI'
tris = [os.path.join(loc_tri,f) for f in os.listdir(loc_tri) if f.endswith(".tif")]

loc_triStdDev = '../EPCExtent_30cm/Elevation_80cmNPS/DSM_TRIStdDev'
os.makedirs(loc_triStdDev, exist_ok=True)

overwrite = True

count = 0
window_sizes = [30]
pairs = {}
for tri in tris:
    for window_size in window_sizes:
        ofile = os.path.join(loc_triStdDev, os.path.basename(tri).replace("_DSMTRI.tif", f"_DSMTRIStdDev{window_size}.tif"))
        pairs[tri] = ofile
        
def calcStdDevWindow(tri, ofile, overwrite=False):
    global count
    s0 = datetime.now()
    count += 1
    if os.path.exists(ofile) and not overwrite:
        return 0
    if "E1020_N430" not in ofile:# and "E0920_N470" not in ofile:
        return
    
    window_size = os.path.basename(ofile).split("_DSMTRIStDev")[1].split(".")[0]
    
    with rio.open(tri) as ras:
        msk = ras.read_masks(1)
        tri_array = ras.read(1)
        tri_array[msk==0] = np.NaN
        kwargs = ras.profile

    stddev_TRI = getStdDevWindow(tri_array, window_size)

    with rio.open(ofile, 'w', **kwargs) as oras:
        oras.write(stddev_TRI, 1)                 
    s1 = datetime.now()
    elapsed = s1-s0
    print(f"\t{elapsed}")
    print(f"{count} - Finished with {ofile}")

    return elapsed
        
time_lapse = Parallel(n_jobs=6)(delayed(calcStdDevWindow)(k, v) for k,v in pairs.items())

In [ ]:
from skimage.morphology import binary_erosion, remove_small_objects

In [ ]:
def developOverviews(file):
    os.system(f"gdalinfo -mm {file}")
    os.system(f"gdaladdo -clean {file}")
    os.system(f"gdaladdo -r average -ro --config COMPRESS_OVERVIEW LZW -minsize 8 {file}")
    
def fillTRIHoles(tri, odir, routes_pc, threshold=1, disk_size=2, overwrite=False):
    ofile = os.path.join(odir, os.path.basename(tri).replace(".tif", f"_FillT{threshold}DS{disk_size}.tif"))
    if os.path.exists(ofile) and not overwrite:
        return ofile
    
    with rio.open(tri) as ras:
        tri_data = ras.read(1)
        mask = ras.read_masks(1)
        kwargs = ras.profile
        rbs= ras.bounds
        bb= box(rbs.left,rbs.bottom, rbs.right, rbs.top)
        res = ras.res[0]
        
    #secondPercentile = np.percentile(test[mask!=0],2)
    #ninetyeightPercentile = np.percentile(test[mask!=0],98)
    #edge_limit = ninetyeightPercentile-((ninetyeightPercentile-secondPercentile)/2)
    #clip and buffer by 10 feet
    routes_local = gpd.clip(routes_pc,bb).buffer(10/res)
    
    # threshold edges to yes or no
    edges = np.where(tri_data <= threshold, 0, 1).astype(np.uint8)
    
    #close 
    edges = closing(edges, disk(disk_size))
    
    # fill regions completely enclosed
    filled = ndi.binary_fill_holes(edges)
    # Erode filled holes
    filled = binary_erosion(filled, selem=square(5))
    #burn in roads as 0 TRI
    if len(routes_local)!=0:
        geom = routes_local.geometry.to_list()
        roads_raster = rasterize(geom, out_shape=(kwargs['width'], kwargs['height']), all_touched=True, transform=kwargs['transform'], fill=1, default_value=0)
        filled = np.minimum(filled,roads_raster)
    
    # remove small regions of isolated edges. Regions smaller than the smallest window used in DTM generation do not need to exist
    #filled = remove_small_objects(filled.astype(bool), min_size=pi*(13*13), connectivity=0)

    edges[mask==0] = 1

    kwargs.update(dtype=np.uint8, nodata=255)
    with rio.open(ofile, 'w', **kwargs) as dst:
        dst.write(filled.astype(np.uint8),1)
    #with rio.open("./roads.tif", 'w', **kwargs) as dst:
    #    dst.write(roads_raster.astype(np.uint8), 1)
        
    developOverviews(ofile)
    
    return ofile

In [ ]:
#fillTRIHoles(ofile, "./", routes_pc, threshold=1, disk_size=1, overwrite=True)
fillTRIHoles('../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm_TRI/E0980_N470_DSM80cm_TRI.tif', "./", routes_pc, threshold=1, disk_size=0, overwrite=True)

In [ ]:
def buildAreaRaster(tri_fill, out_loc, overwrite=False):
    ofile = os.path.join(out_loc, os.path.basename(tri_fill).replace(".tif","_area.tif"))
    if os.path.exists(ofile) and overwrite == False:
        return ofile
    with rio.open(tri_fill) as src:
        array = src.read(1)
    labels, lcount = label(array, return_num=True)
    props = regionprops(labels, array, properties=['label','area'])
    value_map_area = {}
    for i, r in enumerate(props):
        value_map_area[r.label] = r.area
    value_map_area[0] = 0
    area_array = np.vectorize(value_map_area.__getitem__)(labels)
    kwargs.update(dtype=np.uint32)
    with rio.open(ofile, 'w', **kwargs) as dst:
        dst.write(area_array.astype(np.uint32),1)
    return ofile

-------------

In [ ]:
tri_loc = '../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm_TRI'
triFilled_loc = '../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm_TRI_filled'
os.makedirs(triFilled_loc, exist_ok=True)
dsm_tri = [os.path.join(tri_loc,f) for f in os.listdir(tri_loc) if f.endswith(".tif")]
# TRI value is the mean of the difference of a cell and the cells surrounding it
#tri_filled = Parallel(n_jobs=10, verbose=5)(delayed(fillTRIHoles)(tri_file, triFilled_loc, routes_pc, threshold=1, disk_size=0, overwrite=False) for tri_file in dsm_tri)
filledVRT_cmd = """gdalbuildvrt ../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm_TRI_filled/TRI80cmFilled_2019.vrt ../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm_TRI_filled/*.tif"""
os.system(filledVRT_cmd)
print("FINISHED")

In [ ]:
area_loc = "../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm_TRI_filled_Area"
os.makedirs(area_loc, exist_ok==True)
#area_files = Parallel(n_jobs=10, verbose=5)(delayed(buildAreaRaster)(tf, area_loc, overwrite=False) for tf in tri_filled)
areaVRT_cmd = f"""gdalbuildvrt {area_loc}/TRIFilledArea80cm_2019.vrt {area_loc}/*.tif"""
os.system(filledVRT_cmd)
print("FINISHED")

____________../EPCExtent_30cm/________

-----------------------
# Watershed Segmentation of HAG
https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_watershed.html#sphx-glr-auto-examples-segmentation-plot-watershed-py

# Distance Calculation

In [ ]:
def calculateDistance(hag, out_dir, threshold=4, overwrite=False):
    ofile = os.path.join(out_loc, os.path.basename(hag).replace(".tif","_distance.tif"))
    
    if os.path.exists(ofile) and overwrite==False:
        return ofile
    
    with rio.open(hag) as src:
        kwargs = src.profile
        hag_a = src.read(1)
    hag_a = np.where(hag_a>=threshold,1,0)
    
    #calculate euclidean distance to ground
    distance = ndi.distance_transform_edt(hag_a)
    distance = np.ceil(distance)
    
    kwargs.update(dtype=np.int32)
    with rio.open(ofile, 'w', **kwargs) as dst:
        dst.write(distance.astype(np.int32),1)
        
    return ofile

dist_loc = "../EPCExtent_30cm/Elevation_80cmNPS/HAG_NED_80cm_DTG"
os.makedirs(dist_loc, exist_ok==True)
distance_files = Parallel(n_jobs=12, verbose=20)(delayed(calculateDistance)(file, dist_loc, threshold=4, overwrite=False) for file in hag_files)

In [ ]:
import os
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from datetime import datetime
from rasterio.plot import show
from skimage.measure import regionprops, label

t1 = datetime.now()
hag = '../EPCExtent_30cm/Elevation_80cmNPS/HAG_NED_80cm/E1000_N450_DSM80cm_HAG.tif'

with rio.open(hag) as src:
    kwargs = src.profile
    hag_a = src.read(1)
    hag_a = np.where(hag_a>=4,1,0)
    res = src.res[0]
    
# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(hag_a)
local_maxi = peak_local_max(distance, indices=False, labels=hag_a, footprint=np.ones((1, 1)) )
markers = ndi.label(local_maxi)[0]

labels = watershed(-distance, markers, mask=hag_a, compactness=100)

props = regionprops(labels, labels)#, properties=['label','area'])
value_map_area = {}
for i, r in enumerate(props):
    value_map_area[r.label] = r.area
value_map_area[0] = 0
area_array = np.vectorize(value_map_area.__getitem__)(labels)
areaft_array = area_array*(res**2)
    
kwargs.update(dtype=np.int32)
with rio.open("area.tif", 'w', **kwargs) as dst:
    dst.write(areaft_array.astype(np.int32),1)
    
    
t2=datetime.now()
print(t2-t1)

In [ ]:
pdal_extract_pipeline = """[
    %s,
    {
      "type":"filters.colorization",
      "raster":%s,
      "dimension":"DTG"
    },
    "coloured-striped.las"
]"""

________________________